In [1]:
import pandas as pd

df = pd.read_csv('Data/generated_data.csv')

,json,text,label
0,"{'departure_city': 'Олбани', 'arrival_city': '...",Я хочу купить билет в Олбани в Днестровск 18-...,самолет
1,"{'departure_city': 'Порт-Честер', 'arrival_cit...",Я хочу купить билет на рейс из Порт-Честер в ...,самолет
2,"{'departure_city': 'Пейдж-Манор', 'arrival_cit...",Я хочу купить билет из Пейдж-Манора в Ленингр...,самолет
3,"{'departure_city': 'Шлиссельбург', 'arrival_ci...",Я живу в Шлиссельбурге и хочу купить билет в ...,самолет
4,"{'departure_city': 'Згерз', 'arrival_city': 'К...","Я хочу купить билет в Згерз - Коммак, с 25-04...",самолет
...,...,...,...
1995,"{'city': '', 'hotel': 'Villa Amelia', 'date': ...",Я хотел бы забронировать номер в отеле Villa ...,отель
1996,"{'city': 'Северное', 'hotel': 'Rosenalp Gesund...","""Здравствуйте, мне нужно забронировать номер ...",отель
1997,"{'city': 'Хиполуксо', 'hotel': 'Baccaiano Apar...","\nЗдравствуйте, я хотел бы забронировать номер...",отель
1998,"{'city': '', 'hotel': 'The Sun Inn', 'date': '...","""Здравствуйте, я хотел бы забронировать номер...",отель


# Тестирование in-context learning для Mistral-7B-Instruct-v0.2

In [2]:
from vllm import LLM, SamplingParams
from tqdm import tqdm

llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.2")

INFO 04-04 16:56:17 llm_engine.py:70] Initializing an LLM engine with config: model='mistralai/Mistral-7B-Instruct-v0.2', tokenizer='mistralai/Mistral-7B-Instruct-v0.2', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)
INFO 04-04 16:56:28 llm_engine.py:275] # GPU blocks: 12691, # CPU blocks: 2048
INFO 04-04 16:56:28 model_runner.py:501] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-04 16:56:28 model_runner.py:505] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode.
INFO 04-04 16:56:31 model_runner.py:547] Graph capturing finished in 3 se

## Zero-shot learning

In [8]:
prediction = []
sampling_params = SamplingParams(temperature=0, max_tokens=30, stop=['отель', 'самолет'], include_stop_str_in_output=True)
for i in tqdm(range(2000)):
    sample = df.iloc[i]
    prompt = f"""Ты должен отнести текст к одному из двух классов: самолет и отель.
    
    Используй следующий шаблон:
    
    Текст: сообщение пользователя в чате
    Класс: одно из двух значений: самолет или отель
    
    Начинай!
    
    Текст: {sample.text}
    Класс: """
    
    output = llm.generate(prompt, sampling_params, use_tqdm=False)[0]
        
    # Print the outputs.
    prompt = output.prompt
    generated_text = output.outputs[0].text.replace(' ','').replace('\n','')
    prediction.append(generated_text)

100%|██████████| 2000/2000 [02:03<00:00, 16.26it/s]


In [10]:
df['prediction'] = prediction
booking_errors = df[(df['prediction']!=df['label'])&(df['label']=='отель')].shape[0]
ticket_errors = df[(df['prediction']!=df['label'])&(df['label']=='самолет')].shape[0]
print(f'Accuracy booking = {1-booking_errors/1000:.3f}\nAccuracy tickets = {1-ticket_errors/1000:.3f}')

Accuracy booking = 1.000
Accuracy tickets = 0.996


## One-shot learning

In [12]:
prediction_one_shot = []
sampling_params = SamplingParams(temperature=0, max_tokens=30, stop=['отель', 'самолет'], include_stop_str_in_output=True)
for i in tqdm(range(2000)):
    sample = df.iloc[i]
    prompt = f"""Ты должен отнести текст к одному из двух классов: самолет и отель.

    Пример для класса самолет: {df.iloc[0].text}
    Пример для класса отель: {df.iloc[1000].text}
    
    Используй следующий шаблон:
    
    Текст: сообщение пользователя в чате
    Класс: одно из двух значений: самолет или отель
    
    Начинай!
    
    Текст: {sample.text}
    Класс: """
    
    output = llm.generate(prompt, sampling_params, use_tqdm=False)[0]
        
    # Print the outputs.
    prompt = output.prompt
    generated_text = output.outputs[0].text.replace(' ','').replace('\n','')
    prediction.append(generated_text)
    prediction_one_shot.append(generated_text)

100%|██████████| 2000/2000 [02:58<00:00, 11.23it/s]


In [13]:
df['prediction_one_shot'] = prediction_one_shot
booking_errors = df[(df['prediction_one_shot']!=df['label'])&(df['label']=='отель')].shape[0]
ticket_errors = df[(df['prediction_one_shot']!=df['label'])&(df['label']=='самолет')].shape[0]
print(f'Accuracy booking = {1-booking_errors/1000:.3f}\nAccuracy tickets = {1-ticket_errors/1000:.3f}')

Accuracy booking = 1.000
Accuracy tickets = 0.999


# Тестирование in-context learning для Mistral-7B v0.1
(перезапустите ядро)

In [1]:
import pandas as pd

df = pd.read_csv('Data/generated_data.csv')

In [2]:
from vllm import LLM, SamplingParams
from tqdm import tqdm

llm = LLM(model="mistralai/Mistral-7B-v0.1")

INFO 04-04 17:06:52 llm_engine.py:70] Initializing an LLM engine with config: model='mistralai/Mistral-7B-v0.1', tokenizer='mistralai/Mistral-7B-v0.1', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)
INFO 04-04 17:07:04 llm_engine.py:275] # GPU blocks: 8636, # CPU blocks: 2048
INFO 04-04 17:07:04 model_runner.py:501] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-04 17:07:04 model_runner.py:505] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode.
INFO 04-04 17:07:07 model_runner.py:547] Graph capturing finished in 3 secs.


## Zero-shot learning

In [8]:
prediction = []
sampling_params = SamplingParams(temperature=0, max_tokens=30, stop=['отель', 'самолет'], include_stop_str_in_output=True)
for i in tqdm(range(2000)):
    sample = df.iloc[i]
    prompt = f"""Ты должен отнести текст к одному из двух классов: самолет и отель.
    
    Используй следующий шаблон:
    
    Текст: сообщение пользователя в чате
    Класс: одно из двух значений: самолет или отель
    
    Начинай!
    
    Текст: {sample.text}
    Класс: """
    
    output = llm.generate(prompt, sampling_params, use_tqdm=False)[0]
        
    # Print the outputs.
    prompt = output.prompt
    generated_text = output.outputs[0].text
    generated_text = ''.join([char for char in generated_text if char.isalpha()])
    prediction.append(generated_text)

100%|██████████| 2000/2000 [02:04<00:00, 16.05it/s]


In [10]:
df['prediction'] = prediction
booking_errors = df[(df['prediction']!=df['label'])&(df['label']=='отель')].shape[0]
ticket_errors = df[(df['prediction']!=df['label'])&(df['label']=='самолет')].shape[0]
print(f'Accuracy booking = {1-booking_errors/1000:.3f}\nAccuracy tickets = {1-ticket_errors/1000:.3f}')

Accuracy booking = 0.999
Accuracy tickets = 0.998


## One-shot learning

In [20]:
prediction_one_shot = []
sampling_params = SamplingParams(temperature=0, max_tokens=30, stop=['отель', 'самолет'], include_stop_str_in_output=True)
for i in tqdm(range(2000)):
    sample = df.iloc[i]
    prompt = f"""Ты должен отнести текст к одному из двух классов: самолет и отель.

    Пример для класса самолет: {df.iloc[0].text}
    Пример для класса отель: {df.iloc[1000].text}
    
    Используй следующий шаблон:
    
    Текст: сообщение пользователя в чате
    Класс: одно из двух значений: самолет или отель
    
    Начинай!
    
    Текст: {sample.text}
    Класс: """
    
    output = llm.generate(prompt, sampling_params, use_tqdm=False)[0]
        
    # Print the outputs.
    prompt = output.prompt
    generated_text = output.outputs[0].text
    generated_text = ''.join([char for char in generated_text if char.isalpha()])
    prediction_one_shot.append(generated_text)

100%|██████████| 2000/2000 [02:59<00:00, 11.15it/s]


In [21]:
df['prediction_one_shot'] = prediction_one_shot
booking_errors = df[(df['prediction_one_shot']!=df['label'])&(df['label']=='отель')].shape[0]
ticket_errors = df[(df['prediction_one_shot']!=df['label'])&(df['label']=='самолет')].shape[0]
print(f'Accuracy booking = {1-booking_errors/1000:.3f}\nAccuracy tickets = {1-ticket_errors/1000:.3f}')

Accuracy booking = 0.997
Accuracy tickets = 1.000
